In [1]:
import pandas as pd
from openml import OpenMLDataset
from openml.datasets import get_dataset as get_openml_dataset
from pandasgui import show
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Fetch dataset and its description from OpenML. Will be cached in ${HOME}/.openml
data: OpenMLDataset = get_openml_dataset(
    dataset_id="GesturePhaseSegmentationProcessed", version=1, error_if_multiple=True, download_data=True
)

x, y, categorical_indicator, attributed_names = data.get_data(
    target=data.default_target_attribute, dataset_format="dataframe"
)

In [ ]:
# Analyze it with Pandas GUI library
show(x)
# show(y)

In [4]:
y = pd.Series(LabelEncoder().fit_transform(y), index=y.index, name=y.name)

In [5]:
test_size = 0.2
validation_size = 0.1

# Split into train/valid/test according to paper
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=test_size, random_state=1, shuffle=True)
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=validation_size / (1.0 - test_size), random_state=1, shuffle=True
)

In [ ]:
from sklearn.dummy import DummyClassifier

from xtime.datasets import Dataset, DatasetMetadata, DatasetSplit
from xtime.estimators import Estimator
from xtime.ml import ClassificationTask, TaskType

dataset = Dataset(
    metadata=DatasetMetadata(
        name="gesture_phase",
        version="NA",
        task=ClassificationTask(type_=TaskType.MULTI_CLASS_CLASSIFICATION, num_classes=5),
    ),
    splits={
        "train": DatasetSplit(x=train_x, y=train_y),
        "valid": DatasetSplit(x=valid_x, y=valid_y),
        "test": DatasetSplit(x=test_x, y=test_y),
    },
)

estimator = Estimator()
estimator.model = DummyClassifier(strategy="prior").fit(dataset.splits["train"].x, dataset.splits["train"].y)

metrics = estimator.evaluate(dataset)
print(metrics)